### Problem
Given signals $s_1$ and $s_2$, can a series of contractions/dilations turn $s_2$ into a constant multiple of $s_1$? It is assumed that no important data is contained in the periphery of $s_1$, $s_2$.

### Representation of contractions/dilations
Each contraction/dilation can be represented by a pair of numbers $(p, s)$, $p$ representing the site of contraction/dilation and $s$ its size ($s>0$: contraction; $s<0$: dilation). These are assumed to have bivariate normal distribution. 
### Prior distributions
$\Sigma_{p,p} = l$ where l is signal length; $\Sigma_{s,s}=t$, where $t$ is match tolerance. $\Sigma_{p,s}$? No idea yet.

In [ ]:
using Plots
using Distributions

In [ ]:
""" Contracts/dilates `signal` at position `p` by
amount `s`."""
function contralate!(signal, p, s)
    l = length(signal)
    if s == 0 || p < 1 || p > l-1 || l+s < 1 || p+s+1 > l
        return signal
    end
    if s > 0 # dilation
        s = min(l-p-1,s)
        signal[p+s+1:end] = signal[p+1:end-s]
        signal[p+1:p+s+1] .= signal[p]
    else # contraction
        s = min(p-1,-s)
        signal[p+1:end-s] = signal[p+s+1:end]
        signal[end-s+1:end] .= signal[end-s]
    end
    signal
end

contralate(signal, p, s) = contralate!(copy(signal), p, s)
# function contralate(signal, muts)
#     s = copy(signal)
    
# end

In [ ]:
a=collect(1:50)
scatter([a,contralate(a,5,4),contralate(a,10,-2),contralate(a,45,10),contralate(a,4,-5)],legend=:bottomright)

In [ ]:
function tryfit(signal, mat, muts)
#     println(muts)
    m = copy(mat)
    mm = @view m[:]
    for i in size(muts, 1)
        contralate!(mm, muts[i,1], muts[i,2]) 
    end
    soln = m \ signal
    residue = norm(m*soln .- signal, 1)
#     println("Solution: $soln, residue: $residue, matrix: $(m')")
    (soln, 1/residue)
end

In [ ]:
mat = Float64[ 0 0 0 1 2 1 0 0 0 0 0 0
               0 0 0 0 0 0 0 1 0 0 1 0 ]'
sig = Float64[ 0,1,2,1,0,0,0,0,0,0,0,0 ]
scatter([sig,mat])

In [ ]:
nmut = 4
ntry = 1000
nsample = 900
σs = 4.0 # mutation size prior σ
l,n = size(mat)
npoints = 200
μx = linspace(0.0,l*n,npoints)
σx = linspace(-l/2,l/2,npoints)
# all parameters priors assumed to be normal
# column 1: p (mutation position)
# column 2: s (mutation size)
μ = hcat(fill(l*n/2, nmut), fill(0.0, nmut))
σ = hcat(fill(float(2l*n), nmut), fill(σs, nmut))

dists = hcat(Truncated.(Normal.(μ[:,1],σ[:,1]),1,l*n), Normal.(μ[:,2],σ[:,2]))
plot(plot(μx,vec([pdf.(d,μx) for d in dists[:,1]])),
    plot(σx,vec([pdf.(d,σx) for d in dists[:,2]])))

In [ ]:
# for iterations = 1:10
    pars=Array{Int,2}[]
    weighted_scores = Array{Float64}(ntry)
    for i = 1:ntry
        muts = round.(Int,rand.(dists))
        push!(pars, muts)
        scores = tryfit(sig,mat,muts)
        priors = pdf.(dists, muts)
        weighted_scores[i] = scores[2]^4*sqrt(reduce(*,priors))
    end

dists = readjust(wsample(pars,weighted_scores,nsample))
# println(collect(zip(pars,weighted_scores)))
# plot(-l:0.1:l,vec([pdf.(d,-l:0.1:l) for d in dists]))
plot(plot(μx,vec([pdf.(d,μx) for d in dists[:,1]])),
    plot(σx,vec([pdf.(d,σx) for d in dists[:,2]])))

In [ ]:
dists

In [ ]:
function readjust(obs)
    [(==(extrema(o[i] for o in obs)...) ? Normal(obs[1][i],0.1) : fit(Normal,[o[i] for o in obs])) for i in CartesianRange(size(obs[1]))]
end

In [ ]:
==(extrema(1 for x in 1:5)...)

In [ ]:
a=plot()
for x in m
    plot!(x.x,x.density)
end
a

In [ ]:
for i = 1:50
    println(wsample([0.2,0.8]))
end

In [ ]:
x=zeros(Int,50)
x.=(()->wsample([0.2,0.8])).()

In [ ]:
x

In [ ]:
wsample([1,2,3],[0.1,0.2,0.1],4)

In [ ]:
rand(3)[4:end-1]